In [2]:
# importing all the imports needed for this phase
# data manipulation
import pandas as pd
import numpy as np

# visuals
import matplotlib.pyplot as plt
import seaborn as sns

# personal import
import created_functions as cf
import env

# sk imports
from sklearn.model_selection import train_test_split

> ACQUIRE

In [3]:
# creating the url and creating SQL query to pull data
url = env.get_db_url('telco_churn')
query = "select * from customers join contract_types using(contract_type_id) join internet_service_types using(internet_service_type_id) join payment_types using(payment_type_id)"

In [4]:
# pulling database using read sql function
df = pd.read_sql(query, url)

In [5]:
# checking the size of the data
df.shape

(7043, 24)

>there are 7043 rows and 24 columns

from this data i can initially see what are some columns that can be important and some that are not

important
> gender
> tenure
> senior
> monthly

unimportant
> payment_type_id
> internet_service_type
> contract_type_id
> customer_id ( theres already an index)  

>PREPARE

In [6]:
# using prep telco to clan data
df =cf.prep_telco(df)

In [7]:
df.shape

(7032,)

In [8]:
# adding 1 for addons and 0 for no addons
df['has_add_ons'] = df[['online_security'
    , 'online_backup', 'device_protection', 'tech_support']].apply(
    lambda row: 1 if any(x == 'Yes' for x in row) else 0, 
    axis=1
)

KeyError: "None of [Index(['online_security', 'online_backup', 'device_protection',\n       'tech_support'],\n      dtype='object')] are in the [index]"

In [9]:
 df[['online_security'
    , 'online_backup', 'device_protection', 'tech_support']]

KeyError: "None of [Index(['online_security', 'online_backup', 'device_protection',\n       'tech_support'],\n      dtype='object')] are in the [index]"

now split the data

In [ ]:
# splitting the data
train, validate, test = cf.splitting_data(df, 'churn')

In [ ]:
train.shape, validate.shape, test.shape

### preprocess

In [ ]:
# droping colums that were just changed
train = train.drop(columns=['online_security', 'online_backup', 'device_protection', 'tech_support'])
validate = validate.drop(columns=['online_security', 'online_backup', 'device_protection', 'tech_support'])
test = test.drop(columns=['online_security', 'online_backup', 'device_protection', 'tech_support'])

In [ ]:
train.head()

In [ ]:
# created a for loop to visualy see the unique values in each columns 
# and added to binary if 2 and multiclass if more
binary=[]
multiclass= []
for column in train.columns:
    unique_values = train[column].unique()
    print(f"{column}': {unique_values}")
    if train[column].nunique() == 2:
        binary.append(column)
    elif train[column].nunique() > 2:
        multiclass.append(column)

In [ ]:
yn=['partner','dependents','phone_service']

now that i got a see which columns are what going to start seperating them to encod

In [ ]:
len(binary)

In [ ]:
len(multiclass)

> working on the multiclass right now

In [ ]:
# switching internet to 1 or 0
train['has_internet'] = train['internet_service_type'].apply(lambda x: 0 if x == 'None' else 1)

In [ ]:
train.head()

In [ ]:
# deleting the internet service
train.pop('internet_service_type')


In [ ]:
# for lines, streaming make them only yes and no to become binary
lines = ['multiple_lines', 'streaming_tv', 'streaming_movies']

In [ ]:
# loop to change all 3 at to 1 and 0
for x in lines:
    train[x] = train[x].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
# switching contract type to monthly or not
train['monthly_contract'] = train['contract_type'].apply(lambda x: 1 if x == 'Month-to-month' else 0)

In [ ]:
# deleting the contract type col
train.pop('contract_type')

now turning oayment type to 1 or 0 if its automatic

In [ ]:
train['automatic_payment'] = train['payment_type'].apply(lambda x: 1 if 'automatic' in x else 0)


In [ ]:
# now deleting the column
train.pop('payment_type')

In [ ]:
# turned the yes and no rows into 1 (yes) and 0 (no)
for x in yn:
    train[x] = train[x].apply(lambda x: 1 if 'Yes' in x else 0)

In [ ]:
train.head(10)

In [ ]:
# creating is male variable
train['is_male'] = train['gender'].apply(lambda x: 1 if 'Male' in x else 0)

In [ ]:
# now delete the gender 
train.pop('gender')

In [ ]:
# turning churn into 1 and 0
train['churn'] = train['churn'].apply(lambda x: 1 if 'Yes' in x else 0)

In [ ]:
print(train.dtypes)

In [ ]:
train.total_charges.astype(float)